In [ ]:
import pandas as pd
Df=pd.read_excel('/home/nye/Documents/XY/ZH/P1/FinialUse_20190528.xlsx')

Df.columns

LocSerN=pd.DataFrame(Df.drop_duplicates('RawFilePath_Updated20190528').groupby(['PatientID','Loc','SerUpdated_20190525','StudyDate_0008_0020']).size()).reset_index()

LocSerN[LocSerN.loc[:,0]>1]

Df[Df.PatientID=='XY_ZH034'].RawFilePath_Updated20190528.values

Df_Checked=pd.read_excel('/home/nye/Documents/XY/ZH/P1/FinialUse_20190528_Checked.xlsx')

Df_Checked.loc[:,'SerUpdated_20190528']=Df_Checked.SerUpdated_20190525

Df_Checked.loc[~Df_Checked.SerError_20190528.isna(),'SerUpdated_20190528']=Df_Checked.SerError_20190528[~Df_Checked.SerError_20190528.isna()]

Df_Checked.loc[:,'Use20190528_01']=1
Df_Checked.loc[Df_Checked.SerUse==0,'Use20190528_01']=0

Df_Checked.loc[~Df_Checked.index.isin(Df_Checked.drop_duplicates('RawFilePath_Updated20190528').index),'Use20190528_01']=0

LocSerN1=pd.DataFrame(Df_Checked[Df_Checked.Use20190528_01==1].groupby(['PatientID','Loc','SerUpdated_20190528']).size()).reset_index()

LocSerN1[LocSerN1.loc[:,0]>1]

Df_Checked.sort_values('StudyDate_0008_0020').drop_duplicates(['PatientID','Loc','SerUpdated_20190528']).shape

Df_Checked.shape

EarliestScanIndex=Df_Checked.sort_values('StudyDate_0008_0020').drop_duplicates(['PatientID','Loc','SerUpdated_20190528']).index

len(EarliestScanIndex)

EarliestScanIndex[0]

Df_Checked.loc[:,'Use20190528_02']=Df_Checked.Use20190528_01
Df_Checked.loc[~Df_Checked.index.isin(EarliestScanIndex),'Use20190528_02']=0


LocSerN2=pd.DataFrame(Df_Checked[Df_Checked.Use20190528_02==1].groupby(['PatientID','Loc','SerUpdated_20190528']).size()).reset_index()

LocSerN2[LocSerN2.loc[:,0]>1]

Df_Checked.to_excel('/home/nye/Documents/XY/ZH/P1/FinialUseChecked_20190528_Use.xlsx')

Df_Checked.columns

Df=Df_Checked[Df_Checked.Use20190528_02==1]
ImagePathCol='ImageFilePath_Updated20190525_Checked'
PatientIDCol='PatientID'
ResultPathRoot='/home2/nye/ZH/N4C/'
import os
from subprocess import call
def N4(i):
    

    N4PDict={}
    
    RawImage=Df.loc[i,ImagePathCol]
    RawFileName=RawImage.split('/')[-1].split('.')[0]
    
#     UsedName=Df.ImageSubGroup[i]#RawImage.split('.')[0].split('/')[-1]
#     ReslicedName='_'.join([RawFileName,'Resliced'])
    
    TruncatedName='_'.join([RawFileName,'Truncated'])
    Truncated_N4CName='_'.join([TruncatedName,'N4C'])
    
    PatientID=Df.loc[i,PatientIDCol]
#     Image=Df.Image[i]
    ResultPath=os.path.join(ResultPathRoot,PatientID)
#     ReslicedImageRaw=os.path.join(ResultPath,RawImage.split('.')[0].split('/')[-1]+'_Resliced.nii.gz')
#     ReslicedImagePath=os.path.join(ResultPath,ReslicedName+'.nii.gz')
    
#     Zoom=','.join(ZoomSize(Image=Image))
#     reslice=call(['/home/nye/Documents/SH_MH_W/reslice.py','-i',RawImage,'-z',Zoom,'-o',ResultPath,'-m','0'])
#     if reslice==0:
#         call(['mv',ReslicedImageRaw,ReslicedImagePath])
    a=call(['/home/nye/Documents/ImageNormalization/N4BiasC_Truncated.sh',RawImage,ResultPath])
    if a==0:
        N4PDict={ImagePathCol:RawImage,PatientIDCol:PatientID,'RawImagePath':RawImage,
                  'TruncatedN4C_Path':os.path.join(ResultPath,Truncated_N4CName+'.nii.gz'),
                  'TruncatedC_Path':os.path.join(ResultPath,TruncatedName+'.nii.gz')}
    else:
        N4PDict={ImagePathCol:RawImage,PatientIDCol:PatientID,'Error':'N4Error'}
#     else:
#         NeedCheck={'Image':Image,'PatientID':PatientID,'Error':'ResliceProcessError'}
    #N4
    return N4PDict
    #Nyul


Df_Checked[Df_Checked.Use20190528_02==1].ImageFilePath_Updated20190525_Checked[0]

Df_Checked[Df_Checked.Use20190528_02==1].shape

N4(0)

# from multiprocessing import Pool
# N4Reulst=list(Pool(processes=25).imap(N4, Df_Checked[Df_Checked.Use20190528_02==1].index))

N4Df=pd.DataFrame(dict(enumerate(N4Reulst))).T
N4Df.to_excel('/home/nye/Documents/XY/ZH/P1/N4ResultDf_20190528.xlsx')

N4Df.head()

DfChecked_N4C=pd.merge(Df_Checked,N4Df.loc[:,['ImageFilePath_Updated20190525_Checked','TruncatedN4C_Path']],
         on='ImageFilePath_Updated20190525_Checked',how='outer')

Df=DfChecked_N4C[DfChecked_N4C.Use20190528_02==1]
from subprocess import call
ResultPathRoot='/home2/nye/ZH/NyulC'
# OutPutRoot='/home/nye/Documents/SH_MH_W/Test/test_NyulC_simpy/'
SerCol='SerUpdated_20190528'
ImagePathCol='TruncatedN4C_Path'
PatientIDCol='PatientID'
def NyulC(i):
    
    ImageSubGroup=Df.loc[:,SerCol]==i
    RawImageList=Df.loc[ImageSubGroup,ImagePathCol].values
#     MaskList=Df.MergedMaskPath[ImageSubGroup]
    ModelPath=os.path.join(ResultPathRoot,i)+'_NyulNormal_Model.npz'
    PatientIDList=Df.loc[ImageSubGroup,PatientIDCol].values
    NyulCResultList=[os.path.join(ResultPathRoot,I,f.split('.')[0].split('/')[-1]+'_NyulNormalized.nii.gz') 
                     for f,I in zip(RawImageList,PatientIDList) ]
    NyulCResultDictList=[{PatientIDCol:ID,
                          ImagePathCol:N4C,
                          SerCol:i,
                          'NyulC_Path':Nyul,
                          'NyulCModel_Path':ModelPath} 
                         for ID,N4C,Nyul in zip(PatientIDList,RawImageList,NyulCResultList)]
# for i in set(Df.ImageSubGroup)-set([None]):
    RawImageListUse='|'.join(RawImageList)
#     MaskListUse='|'.join(MaskList)import pandas as pd
Df=pd.read_excel('/home/nye/Documents/XY/ZH/P1/FinialUse_20190528.xlsx')

Df.columns

LocSerN=pd.DataFrame(Df.drop_duplicates('RawFilePath_Updated20190528').groupby(['PatientID','Loc','SerUpdated_20190525','StudyDate_0008_0020']).size()).reset_index()

LocSerN[LocSerN.loc[:,0]>1]

Df[Df.PatientID=='XY_ZH034'].RawFilePath_Updated20190528.values

Df_Checked=pd.read_excel('/home/nye/Documents/XY/ZH/P1/FinialUse_20190528_Checked.xlsx')

Df_Checked.loc[:,'SerUpdated_20190528']=Df_Checked.SerUpdated_20190525

Df_Checked.loc[~Df_Checked.SerError_20190528.isna(),'SerUpdated_20190528']=Df_Checked.SerError_20190528[~Df_Checked.SerError_20190528.isna()]

Df_Checked.loc[:,'Use20190528_01']=1
Df_Checked.loc[Df_Checked.SerUse==0,'Use20190528_01']=0

Df_Checked.loc[~Df_Checked.index.isin(Df_Checked.drop_duplicates('RawFilePath_Updated20190528').index),'Use20190528_01']=0

LocSerN1=pd.DataFrame(Df_Checked[Df_Checked.Use20190528_01==1].groupby(['PatientID','Loc','SerUpdated_20190528']).size()).reset_index()

LocSerN1[LocSerN1.loc[:,0]>1]

Df_Checked.sort_values('StudyDate_0008_0020').drop_duplicates(['PatientID','Loc','SerUpdated_20190528']).shape

Df_Checked.shape

EarliestScanIndex=Df_Checked.sort_values('StudyDate_0008_0020').drop_duplicates(['PatientID','Loc','SerUpdated_20190528']).index

len(EarliestScanIndex)

EarliestScanIndex[0]

Df_Checked.loc[:,'Use20190528_02']=Df_Checked.Use20190528_01
Df_Checked.loc[~Df_Checked.index.isin(EarliestScanIndex),'Use20190528_02']=0


LocSerN2=pd.DataFrame(Df_Checked[Df_Checked.Use20190528_02==1].groupby(['PatientID','Loc','SerUpdated_20190528']).size()).reset_index()

LocSerN2[LocSerN2.loc[:,0]>1]

Df_Checked.to_excel('/home/nye/Documents/XY/ZH/P1/FinialUseChecked_20190528_Use.xlsx')

Df_Checked.columns

Df=Df_Checked[Df_Checked.Use20190528_02==1]
ImagePathCol='ImageFilePath_Updated20190525_Checked'
PatientIDCol='PatientID'
ResultPathRoot='/home2/nye/ZH/N4C/'
import os
from subprocess import call
def N4(i):
    

    N4PDict={}
    
    RawImage=Df.loc[i,ImagePathCol]
    RawFileName=RawImage.split('/')[-1].split('.')[0]
    
#     UsedName=Df.ImageSubGroup[i]#RawImage.split('.')[0].split('/')[-1]
#     ReslicedName='_'.join([RawFileName,'Resliced'])
    
    TruncatedName='_'.join([RawFileName,'Truncated'])
    Truncated_N4CName='_'.join([TruncatedName,'N4C'])
    
    PatientID=Df.loc[i,PatientIDCol]
#     Image=Df.Image[i]
    ResultPath=os.path.join(ResultPathRoot,PatientID)
#     ReslicedImageRaw=os.path.join(ResultPath,RawImage.split('.')[0].split('/')[-1]+'_Resliced.nii.gz')
#     ReslicedImagePath=os.path.join(ResultPath,ReslicedName+'.nii.gz')
    
#     Zoom=','.join(ZoomSize(Image=Image))
#     reslice=call(['/home/nye/Documents/SH_MH_W/reslice.py','-i',RawImage,'-z',Zoom,'-o',ResultPath,'-m','0'])
#     if reslice==0:
#         call(['mv',ReslicedImageRaw,ReslicedImagePath])
    a=call(['/home/nye/Documents/ImageNormalization/N4BiasC_Truncated.sh',RawImage,ResultPath])
    if a==0:
        N4PDict={ImagePathCol:RawImage,PatientIDCol:PatientID,'RawImagePath':RawImage,
                  'TruncatedN4C_Path':os.path.join(ResultPath,Truncated_N4CName+'.nii.gz'),
                  'TruncatedC_Path':os.path.join(ResultPath,TruncatedName+'.nii.gz')}
    else:
        N4PDict={ImagePathCol:RawImage,PatientIDCol:PatientID,'Error':'N4Error'}
#     else:
#         NeedCheck={'Image':Image,'PatientID':PatientID,'Error':'ResliceProcessError'}
    #N4
    return N4PDict
    #Nyul


Df_Checked[Df_Checked.Use20190528_02==1].ImageFilePath_Updated20190525_Checked[0]

Df_Checked[Df_Checked.Use20190528_02==1].shape

N4(0)

# from multiprocessing import Pool
# N4Reulst=list(Pool(processes=25).imap(N4, Df_Checked[Df_Checked.Use20190528_02==1].index))

N4Df=pd.DataFrame(dict(enumerate(N4Reulst))).T
N4Df.to_excel('/home/nye/Documents/XY/ZH/P1/N4ResultDf_20190528.xlsx')

N4Df.head()

DfChecked_N4C=pd.merge(Df_Checked,N4Df.loc[:,['ImageFilePath_Updated20190525_Checked','TruncatedN4C_Path']],
         on='ImageFilePath_Updated20190525_Checked',how='outer')

Df=DfChecked_N4C[DfChecked_N4C.Use20190528_02==1]
from subprocess import call
ResultPathRoot='/home2/nye/ZH/NyulC'
# OutPutRoot='/home/nye/Documents/SH_MH_W/Test/test_NyulC_simpy/'
SerCol='SerUpdated_20190528'
ImagePathCol='TruncatedN4C_Path'
PatientIDCol='PatientID'
def NyulC(i):
    
    ImageSubGroup=Df.loc[:,SerCol]==i
    RawImageList=Df.loc[ImageSubGroup,ImagePathCol].values
#     MaskList=Df.MergedMaskPath[ImageSubGroup]
    ModelPath=os.path.join(ResultPathRoot,i)+'_NyulNormal_Model.npz'
    PatientIDList=Df.loc[ImageSubGroup,PatientIDCol].values
    NyulCResultList=[os.path.join(ResultPathRoot,I,f.split('.')[0].split('/')[-1]+'_NyulNormalized.nii.gz') 
                     for f,I in zip(RawImageList,PatientIDList) ]
    NyulCResultDictList=[{PatientIDCol:ID,
                          ImagePathCol:N4C,
                          SerCol:i,
                          'NyulC_Path':Nyul,
                          'NyulCModel_Path':ModelPath} 
                         for ID,N4C,Nyul in zip(PatientIDList,RawImageList,NyulCResultList)]
# for i in set(Df.ImageSubGroup)-set([None]):
    RawImageListUse='|'.join(RawImageList)
#     MaskListUse='|'.join(MaskList)
    Nyul_result=call(['/home/nye/Documents/ImageNormalization/NyulC_simpy.py',
          '--RawImageFiles',RawImageListUse,
#           '--RawImageMaskFiles',MaskListUse,
          '--OutputModelRoot_WithPrefixed',os.path.join(ResultPathRoot,i),
          '--OutputUseRoot',ResultPathRoot,
          '--Run','TrainAndTest','--UseMask','None'])
    if Nyul_result!=0:
        NyulCResultDictList={SerCol:i,'Error':'NyulCError'}
    
        
    return NyulCResultDictList

set(DfChecked_N4C[DfChecked_N4C.Use20190528_02==1].SerUpdated_20190528)

# from multiprocessing import Pool
NyulCReulst=list(Pool(processes=3).imap(NyulC, ['T1', 'T1C', 'T2']))

NyulCReulst[0]

#change the his bin from 10 to 4 
# if use 10 error show up
T1=NyulC('T1')

len([T1]+NyulCReulst[1:])

from itertools import chain
NyulCDf=pd.DataFrame(dict(enumerate(list(chain.from_iterable([T1]+NyulCReulst[1:]))))).T

NyulCDf.head()

DfChecked_N4CNyul=pd.merge(DfChecked_N4C,NyulCDf.loc[:,['NyulCModel_Path','TruncatedN4C_Path','NyulC_Path']],
         on='TruncatedN4C_Path',how='outer')

DfChecked_N4CNyul.shape

DfChecked_N4CNyul.to_excel('/home/nye/Documents/XY/ZH/P1/DfChecked_N4CNyul_20190529.xlsx')
    Nyul_result=call(['/home/nye/Documents/ImageNormalization/NyulC_simpy.py',
          '--RawImageFiles',RawImageListUse,
#           '--RawImageMaskFiles',MaskListUse,
          '--OutputModelRoot_WithPrefixed',os.path.join(ResultPathRoot,i),
          '--OutputUseRoot',ResultPathRoot,
          '--Run','TrainAndTest','--UseMask','None'])
    if Nyul_result!=0:
        NyulCResultDictList={SerCol:i,'Error':'NyulCError'}
    
        
    return NyulCResultDictList

set(DfChecked_N4C[DfChecked_N4C.Use20190528_02==1].SerUpdated_20190528)

# from multiprocessing import Pool
NyulCReulst=list(Pool(processes=3).imap(NyulC, ['T1', 'T1C', 'T2']))

NyulCReulst[0]

#change the his bin from 10 to 4 
# if use 10 error show up
T1=NyulC('T1')

len([T1]+NyulCReulst[1:])

from itertools import chain
NyulCDf=pd.DataFrame(dict(enumerate(list(chain.from_iterable([T1]+NyulCReulst[1:]))))).T

NyulCDf.head()

DfChecked_N4CNyul=pd.merge(DfChecked_N4C,NyulCDf.loc[:,['NyulCModel_Path','TruncatedN4C_Path','NyulC_Path']],
         on='TruncatedN4C_Path',how='outer')

DfChecked_N4CNyul.shape

DfChecked_N4CNyul.to_excel('/home/nye/Documents/XY/ZH/P1/DfChecked_N4CNyul_20190529.xlsx')